# 1. Read .doc, docx file

In [1]:
!pip install --upgrade --quiet python-docx

In [2]:
from docx import Document

# 워드 파일 열기
doc = Document('sample-docs.docx')

# 단락 읽기
for para in doc.paragraphs:
    print(para.text)

Dear shareholders:
As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.
While there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the conditions in which Amazon 

# 2. Invoke LLM using langchain

## 2.1 Install dependencies

In [3]:
!pip install --upgrade --quiet langchain_aws
!pip install --upgrade --quiet botocore boto3

## 2.2 Make LLM

In [4]:
#from langchain_community.chat_models import BedrockChat
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

In [5]:
region_name = "us-west-2"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_kwargs =  { 
    "max_tokens": 4096,
    "temperature": 0.9,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}
llm = ChatBedrock(
    region_name=region_name,
    model_id=model_id,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=model_kwargs
)

## 2.3 Option1. Invoke LLM with Normal Text

In [6]:
prompt = "당신은 영어를 한국어로 번역하는 전문 번역가입니다. I love programming."
messages = [
    HumanMessage(
        content=prompt
    )
]
result = llm(messages)
result.content

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


나는 프로그래밍을 사랑합니다.

'나는 프로그래밍을 사랑합니다.'

## 2.4 Option2. Invoke LLM with Prompt Template

In [7]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Make ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="당신은 영어를 한국어로 번역하는 전문 번역가입니다."),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

# Make Messages List
messages = prompt_template.format_messages(text='I love programming.')

result = llm(messages)
result.content

저는 프로그래밍을 좋아합니다.

'저는 프로그래밍을 좋아합니다.'

## 2.5 Option3. Invoke LLM with Prompt Template

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Make SystemMessagePromptTemplate
system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    "당신은 {input_language}를 {output_language}로 번역하는 전문 번역가입니다."
)

# Make HumanMessagePromptTemplate
human_message_prompt_template = HumanMessagePromptTemplate.from_template("{text}")

# Make ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(
    [
        system_message_prompt_template,
        human_message_prompt_template
    ]
)

# Make Messages List
messages = prompt_template.format_messages(input_language="영어", output_language="한국어", text="I love programming.")

result = llm(messages)
result.content

프로그래밍을 좋아합니다.

'프로그래밍을 좋아합니다.'

## 2.6 Option3. Invoke LLM with Prompt Template and Chain

In [9]:
# Make ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="당신은 영어를 한국어로 번역하는 전문 번역가입니다."),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

# Make Chain
chain = prompt_template | llm

text='I love programming.'
try:
    result = chain.invoke(
        {
            "text": text
        }
    )
    print(result.content)
except Exception:
    err_msg = traceback.format_exc()
    print('error message: ', err_msg)
    raise Exception("Not able to request to LLM")

저는 프로그래밍을 사랑합니다.저는 프로그래밍을 사랑합니다.


# 3. Summarlization using langchain

## 3.1 Install additional dependencies

In [10]:
!pip install --upgrade --quiet anthropic
!pip install --upgrade --quiet pdfplumber
!pip install --upgrade --quiet tiktoken

## 3.2 Quickstart

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

chain = load_summarize_chain(llm, chain_type="map_reduce")

chain.run(docs)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


This is a detailed overview of how to build autonomous agents powered by large language models (LLMs). It covers the key components required: 1) Planning capabilities like task decomposition and self-reflection, 2) Memory management with short-term and long-term memory stores, 3) Tool integration to augment the LLM with external data sources and APIs. It discusses techniques like chain of thought prompting, retrieving relevant memories, and allowing the LLM to call external tools. The post also covers case studies in scientific discovery, generative simulation environments, and proof-of-concept demos like AutoGPT. Finally, it highlights the key challenges like finite context length, limited long-term planning abilities, and the reliability of the natural language interface.This overview discusses building autonomous agents using large language models (LLMs) by incorporating key components: planning capabilities, memory management, and tool integration. It explores techniques like chain

'This overview discusses building autonomous agents using large language models (LLMs) by incorporating key components: planning capabilities, memory management, and tool integration. It explores techniques like chain of thought prompting, memory retrieval, and calling external tools. Case studies in scientific discovery, simulation environments, and demos like AutoGPT are covered. Challenges like context length limitations, long-term planning issues, and reliability of natural language interfaces are highlighted.'

## 3.3 Map-Reduce

In [12]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain.chains.llm import LLMChain

# Map
map_template = """아래 내용에서 주요 부분을 요약해주세요.
{docs}
출력은 항상 <MAP></MAP> 태그 안에 넣어줘.
서두는 작성하지 말아줘.
"""

# Make PromptTemplate
map_prompt = PromptTemplate.from_template(map_template)

# Make LLMChain
# - LLMChain은 초기 처리 단계로, 특정 프롬프트를 사용하여 언어 모델(llm)을 실행
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [13]:
# Reduce
reduce_template = """아래 내용은 문서의 요약본입니다.
{docs}
위 요약본을 이용해서 주요 주제에 대한 최종 통합 요약으로 정제해주세요.
출력은 항상 <REDUCE></REDUCE> 태그 안에 넣어줘.
서두는 작성하지 말아줘.
"""

# Make PromptTemplate
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Make LLMChain
# - LLMChain은 초기 처리 단계로, 특정 프롬프트를 사용하여 언어 모델(llm)을 실행
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

In [14]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
# - StuffDocumentsChain은 여러 문서를 하나의 문자열로 결합하여 LLMChain에 전달하는 역할

# - 이 체인은 문서 목록을 가져와서 먼저 단일 문자열로 결합합니다.
# - 이는 각 문서를 document_prompt를 사용하여 문자열로 포맷한 다음 document_separator를 사용하여 결합함으로써 이를 수행합니다.
# - 그런 다음 document_variable_name으로 설정된 변수 이름을 사용하여 입력에 해당 새 문자열을 추가합니다.
# - 그런 다음 해당 입력은 llm_chain으로 전달됩니다.

# - https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.StuffDocumentsChain.html

combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
# - ReduceDocumentsChain은 문서들을 결합하고, 지정된 토큰 수(token_max)를 초과하지 않도록 반복적으로 축소하는 과정을 담당.
# - 이 과정에서, 문서들이 StuffDocumentsChain의 컨텍스트를 초과할 경우, 동일한 체인(collapse_documents_chain)을 사용하여 처리합니다.

# - 문서를 반복적으로 줄여 문서를 결합합니다.

# - combine_documents_chain 은 항상 제공됩니다. 이것이 호출되는 최종 체인입니다.
# - 이전 결과를 모두 이 체인에 전달하고 이 체인의 출력이 최종 결과로 반환됩니다.

# - collapse_documents_chain은 전달된 문서가 너무 많아서 한 번에 combine_documents_chain에 모두 전달할 수 없는 경우에 사용됩니다.
# - 이 경우,collapse_documents_chain은 허용되는 만큼의 문서 그룹에 대해 재귀적으로 호출됩니다.

reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=50000,
)

In [15]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)



In [16]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from itertools import zip_longest

loader = PDFPlumberLoader("22-26_국제 신재생에너지 정책변화 및 시장 분석.pdf")
docs = loader.load()

def merge_page_content(lst, number):
    result = []
    for i in range(0, len(lst), number):
        page_content = ''
        for j in range(number):
            if i + j < len(lst):
                page_content = page_content + lst[i+j].page_content
        lst[i].page_content = page_content
        result.append(lst[i])
    return result

merge_docs = merge_page_content(docs, 15)
print(len(merge_docs))

9


In [ ]:
# Using Splliter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100, chunk_overlap=0
# )

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=10000,
#     chunk_overlap=50,
#     separators=["\n\n", "\n", "(?<=\. )", " ", ""],
#     length_function=len,
# )

# split_docs = text_splitter.split_documents(docs)
# print(len(split_docs))

In [17]:
print()

print("Final Summarize")
print(map_reduce_chain.run(merge_docs))

<MAP>

아래 내용을 요약하면 다음과 같습니다.

1. 이 연구는 전세계 재생에너지 시장 동향과 정책 변화를 파악하고, 실효성 있는 재생에너지 보급제도를 분석하여 시사점을 도출하는 것이 목적입니다.

2. 전력부문에서 재생에너지 보급이 지속되고 있지만, 1차 에너지 비중 증가는 더딘 추세입니다. 우리나라는 태양광, 풍력 등 변동성 재생에너지 보급에 성과를 보이고 있습니다.

3. 탄소중립 선언과 러시아-우크라이나 전쟁으로 인해 세계적으로 재생에너지 정책이 강화되고 있습니다. 주요국의 탄소중립 법제화 및 재생에너지 정책을 검토하였습니다.

4. 재생에너지 보급제도(FiT/FiP, RPS, 경매제도)의 효과를 실증 분석한 결과, 경매제도의 보급 효과가 가장 높고 RPS 제도의 효과는 제한적인 것으로 나타났습니다.

5. 실증분석 결과를 바탕으로 향후 재생에너지 보급을 위한 제도 개선의 필요성을 제시하고 있습니다.

</MAP>알겠습니다. <MAP>

1.3. 재생에너지원별 균등화발전비용(LCOE)

IRENA(2022)에 따르면 2021년에는 원자재와 물류비용 상승 등의 이슈가 있었지만, 태양광, 육상풍력, 해상풍력 등의 비용 하락 추세는 유지되었다. 태양광은 부품 가격이 상승하였지만 균등화발전비용(LCOE) 하락 추세에는 큰 영향을 미치지 못하였고, 육상풍력은 전년대비 이용율이 크게 개선되면서 비용이 하락하였다. 

2021년 LCOE를 보면 태양광은 전년대비 13% 감소하였고, 해상풍력은 13%, 육상풍력은 15% 감소하였지만, CSP는 7% 증가하였다. 재생에너지 비용은 지난 11년 동안 지열과 수력을 제외하고 모두 감소하면서, 재생에너지 모두 화석발전 비용과 비교하여 경쟁력을 갖추었다. LCOE가 가장 많이 하락한 에너지원은 태양광으로, 2010년 $0.417/kWh에서 2021년 $0.048/kWh로 88% 하락하였다. 

2021년 원자재 가격은 상승하였지만 태양광과 육상풍력 LCOE가 하락할 수 있었던 이유는 첫째, 원자재 가격 상승은 2020년 